In [89]:
import numpy as np
import math
import random
import os 
import sys
import pandas as pd 
random.seed(42)

In [90]:
# Meta information
grid_col = 4
grid_row = 5
canvas_width = 500
canvas_height = 500
canvas_area = 17603.53279986302
wirelength = 0.0
wirelength_cost = 0.0
congestion_cost = 0.0
density_cost = 0.0
project_name = 'circuit_training'
block_name = 'toy_gen_example'
routes_per_micron_hor = 70.33
routes_per_micron_ver = 74.51
routes_used_by_macros_hor = 51.79
routes_used_by_macros_ver = 51.79
smoothing_factor = 2
overlap_threshold = 0.004

# connection limiter: not implemented
EDGE_MAX = None

In [91]:
# Node information
hard_macros = 5
hpin_multiplier = 2
hard_macro_pins = hard_macros * hpin_multiplier # 2-pin macros
# uniform hard macro dimension
hard_macro_width = 10
hard_macro_height = 10

soft_macros = 5
spin_multiplier = 2
soft_macro_pins = soft_macros * spin_multiplier # 2-pin macros
# uniform soft macro dimension
soft_macro_width = 1
soft_macro_height = 1

macros = hard_macros + soft_macros
macro_pins = hard_macro_pins + soft_macro_pins

ports = 15
stdcells = 0

In [92]:
# Node location information
# paddings
left_padding = 1
right_padding = 1
top_padding = 1
bottom_padding = 1

""" ports coordinates """
ports_coord = []

# even ports distance
port_distance = (canvas_width - left_padding - right_padding) // (ports - 1)

# ports all located at bottom of canvas
for i in range(0, ports):
    ports_coord.append((int(bottom_padding), int(left_padding + port_distance * i)))

ports_coord = np.array(ports_coord)
# print(ports_coord) 

""" soft_macro coordinates """
soft_macro_coord = random.choices(range(int(left_padding), int(canvas_width - right_padding)), k=2 * soft_macros)
soft_macro_coord = np.array(soft_macro_coord).reshape((soft_macros, 2))
# print(soft_macro_coord)

""" hard_macro coordinates """
hard_macro_coord = random.choices(range(int(left_padding), int(canvas_width - right_padding)), k=2 * hard_macros)
hard_macro_coord = np.array(hard_macro_coord).reshape((hard_macros, 2))
# print(hard_macro_coord)


In [93]:
# directory creation
OUT_DIR = './toy_gen_example/'
OUT_NETLIST = 'netlist.pb.txt'
OUT_PLC = 'initial.plc'

if not os.path.exists(OUT_DIR):  
  # Create a new directory because it does not exist 
  os.makedirs(OUT_DIR)
  print("Directory Created!")

In [94]:
f = open(OUT_DIR + OUT_NETLIST, "w+")
f.write("# proto-file: tensorflow/core/framework/graph.proto\n")
f.write("# proto-message: tensorflow.GraphDef\n")

EDGE_COUNT = 0

def attr_writer(fd,
                key: str,
                value: str) -> None:
    fd.write('  attr {\n')
    fd.write('    key: "' + key + '"\n')
    fd.write('    value {\n')
    fd.write('      ' + value + '\n')
    fd.write('    }\n')
    fd.write('  }\n')

def IO_choice(ifPort = False):
    if random.uniform(0, 1) > 0 or ifPort:
        # Select pins to input
        IO_dict = {'hard':None, 'soft':None, 'port':None}
        IO_dict['hard'] = random.sample(range(0, hard_macros),random.randint(0, hard_macros-1))
        IO_dict['soft'] = random.sample(range(0, soft_macros),random.randint(0, soft_macros-1))
        IO_dict['port'] = random.sample(range(0, ports),random.randint(0, ports-1))
        return IO_dict
    else:
        # Do nothing
        return None

def pin_offset(hard = True):
    if hard:
        flat_pin_range = 2 * hard_macro_height + hard_macro_width
        flat_pin_dist = flat_pin_range / (hpin_multiplier - 1)
        pin_offset = []

        for pin_idx in range(0, hpin_multiplier):
            flat_coord = pin_idx * flat_pin_dist
            
            if flat_coord <= hard_macro_height:
                x_offset = -1 * hard_macro_width / 2

                if flat_coord <= (hard_macro_height / 2):
                    y_offset = hard_macro_height / 2 - flat_coord
                else:
                    y_offset = -1 * (flat_coord - hard_macro_height / 2)

                pin_offset.append((x_offset, y_offset))

            elif flat_coord <= hard_macro_height + hard_macro_width:
                y_offset = -1 * hard_macro_height / 2

                temp_coord = flat_coord - hard_macro_height

                if temp_coord <= (hard_macro_width / 2):
                    x_offset = -1 * (hard_macro_width / 2 - temp_coord)
                else:
                    x_offset = temp_coord - hard_macro_width / 2


                pin_offset.append((x_offset, y_offset))

            elif flat_coord <= flat_pin_range:
                x_offset = hard_macro_width / 2

                temp_coord = flat_coord - (hard_macro_height + hard_macro_width)
                
                if temp_coord <= (hard_macro_height / 2):
                    y_offset = -1 * (hard_macro_height / 2 - temp_coord)
                else:
                    y_offset = temp_coord - hard_macro_height / 2

                pin_offset.append((x_offset, y_offset))
    return pin_offset


for port_idx in range(0, ports):
    f.write('node {\n')
    
    f.write('  name: ' + '"P' + str(port_idx) + '"\n')

    temp_dict = IO_choice(ifPort=True)
    if temp_dict['hard'] is not None:
        for h in temp_dict['hard']:
            f.write('  input: "P'+ str(random.randint(0,hpin_multiplier-1)) + '_M' + str(h) + '"\n')
            EDGE_COUNT += 1

    if temp_dict['soft'] is not None:
        for s in temp_dict['soft']:
            f.write('  input: "Grp_' + str(s) + '/Pinput"\n')
            EDGE_COUNT += 1

    attr_writer(f, 'side', 'placeholder: "BOTTOM"')
    attr_writer(f, 'type', 'placeholder: "PORT"')
    attr_writer(f, 'x', 'f: ' + str(ports_coord[port_idx][0]))
    attr_writer(f, 'y', 'f: ' + str(ports_coord[port_idx][1]))

    f.write('}\n')

for hmacro_idx in range(0, hard_macros):
    f.write('node {\n')

    f.write('  name: ' + '"M' + str(hmacro_idx) + '"\n')
    
    attr_writer(f, 'height', 'f: ' + str(hard_macro_height))
    attr_writer(f, 'orientation', 'placeholder: "N"')
    attr_writer(f, 'type', 'placeholder: "MACRO"')
    attr_writer(f, 'width', 'f: ' + str(hard_macro_width))

    f.write('}\n')

hard_macro_pin_offsets = pin_offset(hard=True)

for hmacro_idx in range(0, hard_macros):
    for pin_idx in range(0, hpin_multiplier):
        f.write('node {\n')
        
        f.write('  name: ' + '"P' + str(pin_idx) + '_M'+ str(hmacro_idx) + '"\n')

        temp_dict = IO_choice()
        if temp_dict['port'] is not None:
            for p in temp_dict['hard']:
                f.write('  input: "P' + str(p) + '"\n')
                EDGE_COUNT += 1

        if temp_dict['soft'] is not None:
            for s in temp_dict['soft']:
                f.write('  input: "Grp_' + str(s) + '/Pinput"\n')
                EDGE_COUNT += 1

        attr_writer(f, 'macro_name', 'placeholder: "M' + str(hmacro_idx) + '"')
        attr_writer(f, 'type', 'placeholder: "MACRO_PIN"')
        attr_writer(f, 'x_offset', 'f: ' + str(int(hard_macro_pin_offsets[pin_idx][0])))
        attr_writer(f, 'y_offset', 'f: ' + str(int(hard_macro_pin_offsets[pin_idx][1])))

        f.write('}\n')

for smacro_idx in range(0, soft_macros):
    f.write('node {\n')

    f.write('  name: ' + '"Grp_' + str(smacro_idx) + '"\n')
    
    attr_writer(f, 'height', 'f: ' + str(soft_macro_height))
    attr_writer(f, 'type', 'placeholder: "macro"')
    attr_writer(f, 'width', 'f: ' + str(soft_macro_width))
    attr_writer(f, 'x', 'f: ' + str(soft_macro_coord[smacro_idx][0]))
    attr_writer(f, 'y', 'f: ' + str(soft_macro_coord[smacro_idx][1]))

    f.write('}\n')

    # poutput
    f.write('node {\n')

    f.write('  name: ' + '"Grp_' + str(smacro_idx) + '/Poutput_single_0' + '"\n')
    temp_dict = IO_choice()
    if temp_dict['port'] is not None:
        for p in temp_dict['hard']:
            f.write('  input: "P' + str(p) + '"\n')
            EDGE_COUNT += 1

    if temp_dict['hard'] is not None:
        for h in temp_dict['hard']:
            f.write('  input: "P'+ str(random.randint(0,hpin_multiplier-1)) + '_M' + str(h) + '"\n')
            EDGE_COUNT += 1

    attr_writer(f, 'macro_name', 'placeholder: ' + '"Grp_' + str(smacro_idx) + '"')
    attr_writer(f, 'type', 'placeholder: "macro_pin"')
    attr_writer(f, 'x', 'f: ' + str(soft_macro_coord[smacro_idx][0]))
    attr_writer(f, 'x_offset', 'f: 0')
    attr_writer(f, 'y', 'f: ' + str(soft_macro_coord[smacro_idx][1]))
    attr_writer(f, 'y_offset', 'f: 0')

    f.write('}\n')

    # pinput
    f.write('node {\n')

    f.write('  name: ' + '"Grp_' + str(smacro_idx)  + '/Pinput' + '"\n')
   
    attr_writer(f, 'macro_name', 'placeholder: ' + '"Grp_' + str(smacro_idx) + '"')
    attr_writer(f, 'type', 'placeholder: "macro_pin"')
    attr_writer(f, 'x', 'f: ' + str(soft_macro_coord[smacro_idx][0]))
    attr_writer(f, 'x_offset', 'f: 0')
    attr_writer(f, 'y', 'f: ' + str(soft_macro_coord[smacro_idx][1]))
    attr_writer(f, 'y_offset', 'f: 0')

    f.write('}\n')
    

f.close()

In [95]:
f = open(OUT_DIR + OUT_PLC, "w+")
f.write('# Placement file for Circuit Training\n')
f.write('# Source input file(s) : circuit_training/environment/test_data/' + block_name + '/netlist.pb.txt\n')
f.write('# This file : circuit_training/environment/test_data/' + block_name + '/initial.plc\n')
f.write('# Columns : ' + str(grid_col) + '  Rows : ' + str(grid_row) + '\n')
f.write('# Width : ' + str(canvas_width) + '  Height : ' + str(canvas_height) + '\n')
f.write('# Area : ' + str(canvas_area) + '\n')
f.write('# Wirelength : ' + str(wirelength) + '\n')
f.write('# Wirelength cost : ' + str(wirelength_cost) + '\n')
f.write('# Congestion cost : ' + str(congestion_cost) + '\n')
f.write('# Density cost : ' + str(density_cost) + '\n')
f.write('# Project : circuit_training\n')
f.write('# Block : toy_gen_example\n')
f.write('# Routes per micron, hor : ' + str(routes_per_micron_hor) + ' ver : ' + str(routes_per_micron_ver) + '\n')
f.write('# Routes used by macros, hor : ' + str(routes_used_by_macros_hor) + ' ver : ' + str(routes_used_by_macros_ver) + '\n')
f.write('# Smoothing factor : ' + str(smoothing_factor) + '\n')
f.write('# Overlap threshold : ' + str(overlap_threshold) + '\n')

f.write('#\n')
f.write('#\n')
f.write('#\n')
f.write('# Counts of node types:\n')
f.write('# HARD_MACROs     :         ' + str(hard_macros) + '\n')
f.write('# HARD_MACRO_PINs :         ' + str(hard_macro_pins) + '\n')
f.write('# MACROs          :         ' + str(macros) + '\n')
f.write('# MACRO_PINs      :         ' + str(macro_pins) + '\n')
f.write('# PORTs           :         ' + str(ports) + '\n')
f.write('# SOFT_MACROs     :         ' + str(soft_macros) + '\n')
f.write('# SOFT_MACRO_PINs :         ' + str(soft_macro_pins) + '\n')
f.write('# STDCELLs        :         ' + str(stdcells) + '\n')
f.write('#\n')
f.write('# node_index x y orientation fixed\n')

index = 0
for idx in range(0, ports):
    f.write(str(index) + ' ' + str(ports_coord[idx][0]) + ' ' + str(ports_coord[idx][1]) + ' - ' + '1\n')
    index += 1

for idx in range(0, hard_macros):
    f.write(str(index) + ' ' + str(hard_macro_coord[idx][0]) + ' ' + str(hard_macro_coord[idx][1]) + ' N ' + '0\n')
    index += 1

index += hard_macro_pins
for idx in range(0, soft_macros):
    f.write(str(index) + ' ' + str(soft_macro_coord[idx][0]) + ' ' + str(soft_macro_coord[idx][1]) + ' N ' + '0\n')
    index += (1 + spin_multiplier)

f.close()

In [96]:
print(EDGE_COUNT)

108
